In [ ]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


#CIFAR_10 set of 60k images 32x32 pixels on 3 channels
IMG_CHANNELS = 3

IMG_ROWS = 32
IMG_COLS = 32

#CONSTANTS
BATCH_SIZE = 128
NB_EPOCH = 20
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print("X_train shape: ", X_train.shape)
print(X_train.shape[0], " train samples")
print(X_test.shape[0], " test samples")


#convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

#float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

#network
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', 
input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))

#save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
model.save_weights('cifar10_weights.h5', overwrite=True)


model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))

model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

#save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
model.save_weights('cifar10_weights.h5', overwrite=True)

from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
import numpy as np
NUM_TO_AUGMENT = 5

#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# augmenting
print("Augmenting training set images...")

datagen = ImageDataGenerator(
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')

xtas, ytas = [], []
for i in range(X_train.shape[0]):
    num_aug = 0
x = X_train[i] # (3, 32, 32)
x = x.reshape((1,) + x.shape) # (1, 3, 32, 32)
for x_aug in datagen.flow(x, batch_size=1, 
save_to_dir='preview', save_prefix='cifar', save_format='jpeg'):
    if num_aug >= NUM_TO_AUGMENT:
        break
xtas.append(x_aug[0])
num_aug += 1


#fit the datagen
datagen.fit(X_train)

#train
history = model.fit_generator(
    datagen.flow(
        X_train, 
        Y_train, 
        batch_size=BATCH_SIZE),
    samples_per_epoch=X_train.shape[0],
    epochs=NB_EPOCH,
    verbose=VERBOSE)

score = model.evaluate(
    X_test,
    Y_test, 
    batch_size=BATCH_SIZE, 
    verbose=VERBOSE)
print("Test score: ", score[0])
print("Test accuracy: ", score[1])

X_train shape:  (50000, 32, 32, 3)
50000  train samples
10000  test samples
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_14 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 16, 16, 32)        0         
____________________________________________

### Ethical and Privacy concerns:
This neural network has many ethical and practical concerns considering how inaccurate it is. Verifying the training data set for this algorithm is varied and unbiased as possible is very important as it could result in not only poor performance but also racial and sexist biases as well. Suppose that the training data was only pictures of only Eastern Asian male faces. The deep neural network would likely have difficulty recognizing Black people and Women. 

If used for something where the face was used to say open a automatic door then many would not be able to gain entrance into the space. Changing gears, suppose the training data set included pictures of real life people who did not consent to having their images used in a training algorithm. While it may not seem like a big deal you could have a situation in which an organization were to use the neural network to try and track large groups of people and report their findings to employers, government agencies, and various service providers. This could result in the person being denied employment, denied service, or inaccurately charged with criminal penalties. Law enforcements use of facial recognition technology (FRT) is concerning  because "such usage is not disclosed, and people are not aware that they are under state survellance. The use of FRT has been marked by criticism of bias, discrimination, and lack of transparency" (Sarabdeen). 

#### References:
Sarabdeen, J. (Mar 2022). Protection of the rights of individuals when using facial recognitiontechnology. Retrieved from: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8927940